# Age Detection of Indian Actors | Analytics Vidhya


### About Problem 
Indian Movie Face database (IMFDB) is a large unconstrained face database consisting of 34512 images of 100 Indian actors collected from more than 100 videos. All the images are manually selected and cropped from the video frames resulting in a high degree of variability interms of scale, pose, expression, illumination, age, resolution, occlusion, and makeup. IMFDB is the first face database that provides a detailed annotation of every image in terms of age, pose, gender, expression and type of occlusion that may help other face related applications. For more details about the data set, read here

Data
The dataset is cleaned and formatted to give you a total of 26742 images with 19906 images in train and 6636 images in test.

The task is to predict the age of a person from his or her facial attributes. For simplicity, the problem has been converted to a multiclass problem with classes as Young, Middle and Old.

The attributes of data are as follows:
ID – Unique ID of image
Class – Age bin of person in image

#### Dataset in below link
https://datahack.analyticsvidhya.com/contest/practice-problem-age-detection/

#### Datasets structure

train
    |-Train (folder contains training all images)
    |-train.csv (Contains image names as ID's and class label which it belongs to)
    
test
    |-Test (folder contains test all images)
    |-test.csv (Contains image names as ID's)

#### Below is the best approach and worked well for this problem and got 88.53 accuracy on test set which put this result in 19th position in this competetion. This can be taken further if more time spent with GPU processing. Need to tweak parameters on the best model till date.

Storing all the file, folder paths in variable for convenience so that we don't need to touch the code if input or output path is changed. Like a configuration file. 

In [33]:
# define the paths to the images directory
IMAGES_PATH = "download/train/Train"
#IMAGES_PATH = "downloads\\train\\Train"
IMAGES_CSV = "download/train/train.csv"
#IMAGES_CSV = "downloads\\train\\train.csv"
TEST_IMAGES_PATH = "download/test/Test"
#TEST_IMAGES_PATH = "downloads\\test\\Test"
TEST_IMAGES_CSV = "download/test/test.csv"

# since we do not have validation data or access to the testing
# labels we need to take a number of images from the training
# data and use them instead
NUM_CLASSES = 3
NUM_VAL_IMAGES = 0.25

# define the path to the output training, validation, and testing
# HDF5 files
TRAIN_HDF5 = "hdf5/train.hdf5"
VAL_HDF5 = "hdf5/val.hdf5"
TEST_HDF5 = "hdf5/test.hdf5"
#TRAIN_HDF5 = "hdf5\\train.hdf5"
#VAL_HDF5 = "hdf5\\val.hdf5"
#TEST_HDF5 = "hdf5\\test.hdf5"

# path to the output model file
MODEL_PATH = "output/alexnet_age_pred.model"
#MODEL_PATH = "output\\alexnet_age_pred.model"

# define the path to the dataset mean
DATASET_MEAN = "output/age_pred_mean.json"
#DATASET_MEAN = "output\\age_pred_mean.json"

# define the path to the output directory used for storing plots,
# classification reports, etc.
OUTPUT_PATH = "output"

For rescaling the images (up/down) with required aspect ratio, so that features in the images will no get that affect. Usually this works well on large datasets. 

In [34]:
# import the necessary packages
import imutils
import cv2

class AspectAwarePreprocessor:
	def __init__(self, width, height, inter=cv2.INTER_AREA):
		# store the target image width, height, and interpolation
		# method used when resizing
		self.width = width
		self.height = height
		self.inter = inter

	def preprocess(self, image):
		# grab the dimensions of the image and then initialize
		# the deltas to use when cropping
		(h, w) = image.shape[:2]
		dW = 0
		dH = 0

		# if the width is smaller than the height, then resize
		# along the width (i.e., the smaller dimension) and then
		# update the deltas to crop the height to the desired
		# dimension
		if w < h:
			image = imutils.resize(image, width=self.width,
				inter=self.inter)
			dH = int((image.shape[0] - self.height) / 2.0)

		# otherwise, the height is smaller than the width so
		# resize along the height and then update the deltas
		# crop along the width
		else:
			image = imutils.resize(image, height=self.height,
				inter=self.inter)
			dW = int((image.shape[1] - self.width) / 2.0)

		# now that our images have been resized, we need to
		# re-grab the width and height, followed by performing
		# the crop
		(h, w) = image.shape[:2]
		image = image[dH:h - dH, dW:w - dW]

		# finally, resize the image to the provided spatial
		# dimensions to ensure our output image is always a fixed
		# size
		return cv2.resize(image, (self.width, self.height),
			interpolation=self.inter)

If we have more number of images then I/O operations will be little costly in time taking and ram memory. So images data can be stored in hdf5 format files but only problem is when the images are converted to hdf5 the size of total dataset will be increased but can load set images at once of a single batch in epoch.

In [35]:
# import the necessary packages
import h5py
import os

class HDF5DatasetWriter:
	def __init__(self, dims, outputPath, dataKey="images",
		bufSize=1000):
		# check to see if the output path exists, and if so, raise
		# an exception
		if os.path.exists(outputPath):
			raise ValueError("The supplied `outputPath` already "
				"exists and cannot be overwritten. Manually delete "
				"the file before continuing.", outputPath)

		# open the HDF5 database for writing and create two datasets:
		# one to store the images/features and another to store the
		# class labels
		self.db = h5py.File(outputPath, "w")
		self.data = self.db.create_dataset(dataKey, dims,
			dtype="float")
		self.labels = self.db.create_dataset("labels", (dims[0],),
			dtype="int")

		# store the buffer size, then initialize the buffer itself
		# along with the index into the datasets
		self.bufSize = bufSize
		self.buffer = {"data": [], "labels": []}
		self.idx = 0

	def add(self, rows, labels):
		# add the rows and labels to the buffer
		self.buffer["data"].extend(rows)
		self.buffer["labels"].extend(labels)

		# check to see if the buffer needs to be flushed to disk
		if len(self.buffer["data"]) >= self.bufSize:
			self.flush()

	def flush(self):
		# write the buffers to disk then reset the buffer
		i = self.idx + len(self.buffer["data"])
		self.data[self.idx:i] = self.buffer["data"]
		self.labels[self.idx:i] = self.buffer["labels"]
		self.idx = i
		self.buffer = {"data": [], "labels": []}

	def storeClassLabels(self, classLabels):
		# create a dataset to store the actual class label names,
		# then store the class labels
		dt = h5py.special_dtype(vlen=str) # `vlen=unicode` for Py2.7
		labelSet = self.db.create_dataset("label_names",
			(len(classLabels),), dtype=dt)
		labelSet[:] = classLabels

	def close(self):
		# check to see if there are any other entries in the buffer
		# that need to be flushed to disk
		if len(self.buffer["data"]) > 0:
			self.flush()

		# close the dataset
		self.db.close()

Creating hfd5 datasets for train, validation and test. Get the image paths and labels and add data to hdf5 file using HDF5DatasetWriter class object.

In [36]:
# import the necessary packages
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imutils import paths
import numpy as np
import progressbar
import json
import cv2
import os

# grab the paths to the images
print("[INFO] loading train csv file to create image paths and labels data...")
f = open(IMAGES_CSV)
f.__next__() # f.next() for Python 2.7

trainPaths = []
trainLabels = []
flist = list(f) 
for i, row in enumerate(flist):
	# extract the image and label from the row
	(image, label) = row.strip().split(",")
	imagepath = os.path.join(IMAGES_PATH, image)
    
	trainPaths.append(imagepath)
	trainLabels.append(label)
f.close()

le = LabelEncoder()
trainLabels = le.fit_transform(trainLabels)

# perform another stratified sampling, this time to build the
# validation data
split = train_test_split(trainPaths, trainLabels,
	test_size=NUM_VAL_IMAGES, stratify=trainLabels,
	random_state=99)
(trainPaths, valPaths, trainLabels, valLabels) = split

testPaths = [] #list(paths.list_images(TEST_IMAGES_PATH))
t = open(TEST_IMAGES_CSV)
t.__next__() # f.next() for Python 2.7
tlist = list(t) 
for i, row in enumerate(tlist):
	# extract the image and label from the row
	image = row.rstrip()
	imagepath = os.path.join(TEST_IMAGES_PATH, image)  
	testPaths.append(imagepath)
t.close()    
testLabels = ["TEST"] * len(testPaths)

# construct a list pairing the training, validation, and testing
# image paths along with their corresponding labels and output HDF5
# files
datasets = [
	("train", trainPaths, trainLabels, TRAIN_HDF5),
	("val", valPaths, valLabels, VAL_HDF5),
#	("test", testPaths, testLabels, TEST_HDF5)
]

# initialize the image pre-processor and the lists of RGB channel
# averages
aap = AspectAwarePreprocessor(256, 256)
(R, G, B) = ([], [], [])

# loop over the dataset tuples
for (dType, paths, labels, outputPath) in datasets:
	# create HDF5 writer
	print("[INFO] building {}...".format(outputPath))
	writer = HDF5DatasetWriter((len(paths), 256, 256, 3), outputPath)
	if dType in ["train","val"]:
		writer.storeClassLabels(le.classes_)
	else:
		labels = le.fit_transform(labels)
		writer.storeClassLabels(le.classes_)

	# initialize the progress bar
	widgets = ["Building Dataset: ", progressbar.Percentage(), " ",
		progressbar.Bar(), " ", progressbar.ETA()]
	pbar = progressbar.ProgressBar(maxval=len(paths),
		widgets=widgets).start()

	# loop over the image paths
	for (i, (path, label)) in enumerate(zip(paths, labels)):
		# load the image and process it
		image = cv2.imread(path)
		image = aap.preprocess(image)

		# if we are building the training dataset, then compute the
		# mean of each channel in the image, then update the
		# respective lists
		if dType == "train":
			(b, g, r) = cv2.mean(image)[:3]
			R.append(r)
			G.append(g)
			B.append(b)

		# add the image and label # to the HDF5 dataset
		writer.add([image], [label])
		pbar.update(i)

	# close the HDF5 writer
	pbar.finish()
	writer.close()

# construct a dictionary of averages, then serialize the means to a
# JSON file
print("[INFO] serializing means...")
D = {"R": np.mean(R), "G": np.mean(G), "B": np.mean(B)}
f = open(DATASET_MEAN, "w")
f.write(json.dumps(D))
f.close()

[INFO] loading train csv file to create image paths and labels data...
[INFO] building hdf5/train.hdf5...


Building Dataset: 100% |########################################| Time: 0:00:47


[INFO] building hdf5/val.hdf5...


Building Dataset: 100% |########################################| Time: 0:00:49


[INFO] serializing means...


Vewing hdf5 datasets

In [ ]:
filenames = ["hdf5/train.hdf5","hdf5/val.hdf5","hdf5/test.hdf5"]
for f in filenames:
    db = h5py.File(f, "r")
    print(db.keys())
    print(db["images"].shape)
    print(db["label_names"].shape)
    print(db["labels"].shape)
    print(db["label_names"][:])
    db.close()

Image preprocessros

In [38]:
# import the necessary packages
from keras.preprocessing.image import img_to_array

class ImageToArrayPreprocessor:
	def __init__(self, dataFormat=None):
		# store the image data format
		self.dataFormat = dataFormat

	def preprocess(self, image):
		# apply the Keras utility function that correctly rearranges
		# the dimensions of the image
		return img_to_array(image, data_format=self.dataFormat)

In [39]:
# import the necessary packages
import cv2

class SimplePreprocessor:
	def __init__(self, width, height, inter=cv2.INTER_AREA):
		# store the target image width, height, and interpolation
		# method used when resizing
		self.width = width
		self.height = height
		self.inter = inter

	def preprocess(self, image):
		# resize the image to a fixed size, ignoring the aspect
		# ratio
		return cv2.resize(image, (self.width, self.height),
			interpolation=self.inter)

In [40]:
# import the necessary packages
from sklearn.feature_extraction.image import extract_patches_2d

class PatchPreprocessor:
	def __init__(self, width, height):
		# store the target width and height of the image
		self.width = width
		self.height = height

	def preprocess(self, image):
		# extract a random crop from the image with the target width
		# and height
		return extract_patches_2d(image, (self.height, self.width),
			max_patches=1)[0]

In [41]:
# import the necessary packages
import cv2

class MeanPreprocessor:
	def __init__(self, rMean, gMean, bMean):
		# store the Red, Green, and Blue channel averages across a
		# training set
		self.rMean = rMean
		self.gMean = gMean
		self.bMean = bMean

	def preprocess(self, image):
		# split the image into its respective Red, Green, and Blue
		# channels
		(B, G, R) = cv2.split(image.astype("float32"))

		# subtract the means for each channel
		R -= self.rMean
		G -= self.gMean
		B -= self.bMean

		# merge the channels back together and return the image
		return cv2.merge([B, G, R])

Extended class for Callback to save the model after each epoch or for number of epochs

In [42]:
# import the necessary packages
from keras.callbacks import Callback
import os

class EpochCheckpoint(Callback):
	def __init__(self, outputPath, every=5, startAt=0):
		# call the parent constructor
		super(Callback, self).__init__()

		# store the base output path for the model, the number of
		# epochs that must pass before the model is serialized to
		# disk and the current epoch value
		self.outputPath = outputPath
		self.every = every
		self.intEpoch = startAt

	def on_epoch_end(self, epoch, logs={}):
		# check to see if the model should be serialized to disk
		if (self.intEpoch + 1) % self.every == 0:
			p = os.path.sep.join([self.outputPath,
				"epoch_{}.hdf5".format(self.intEpoch + 1)])
			self.model.save(p, overwrite=True)

		# increment the internal epoch counter
		self.intEpoch += 1

Extended class for BaseLogger to save the training metrics and generate plot to visualize the training progress.

In [43]:
# import the necessary packages
from keras.callbacks import BaseLogger
import matplotlib.pyplot as plt
import numpy as np
import json
import os

class TrainingMonitor(BaseLogger):
	def __init__(self, figPath, jsonPath=None, startAt=0):
		# store the output path for the figure, the path to the JSON
		# serialized file, and the starting epoch
		super(TrainingMonitor, self).__init__()
		self.figPath = figPath
		self.jsonPath = jsonPath
		self.startAt = startAt

	def on_train_begin(self, logs={}):
		# initialize the history dictionary
		self.H = {}

		# if the JSON history path exists, load the training history
		if self.jsonPath is not None:
			if os.path.exists(self.jsonPath):
				self.H = json.loads(open(self.jsonPath).read())

				# check to see if a starting epoch was supplied
				if self.startAt > 0:
					# loop over the entries in the history log and
					# trim any entries that are past the starting
					# epoch
					for k in self.H.keys():
						self.H[k] = self.H[k][:self.startAt]

	def on_epoch_end(self, epoch, logs={}):
		# loop over the logs and update the loss, accuracy, etc.
		# for the entire training process
		for (k, v) in logs.items():
			l = self.H.get(k, [])
			l.append(float(v))
			self.H[k] = l

		# check to see if the training history should be serialized
		# to file
		if self.jsonPath is not None:
			f = open(self.jsonPath, "w")
			f.write(json.dumps(self.H))
			f.close()

		# ensure at least two epochs have passed before plotting
		# (epoch starts at zero)
		if len(self.H["loss"]) > 1:
			# plot the training loss and accuracy
			N = np.arange(0, len(self.H["loss"]))
			plt.style.use("ggplot")
			plt.figure()
			plt.plot(N, self.H["loss"], label="train_loss")
			plt.plot(N, self.H["val_loss"], label="val_loss")
			plt.plot(N, self.H["accuracy"], label="train_acc")
			plt.plot(N, self.H["val_accuracy"], label="val_acc")
			plt.title("Training Loss and Accuracy [Epoch {}]".format(
				len(self.H["loss"])))
			plt.xlabel("Epoch #")
			plt.ylabel("Loss/Accuracy")
			plt.legend()

			# save the figure
			plt.savefig(self.figPath)
			plt.close()

To get the input data as a set of batchs for training or testing. 

In [44]:
# import the necessary packages
from keras.utils import np_utils
import numpy as np
import h5py

class HDF5DatasetGenerator:
	def __init__(self, dbPath, batchSize, preprocessors=None,
		aug=None, binarize=True, classes=3):
		# store the batch size, preprocessors, and data augmentor,
		# whether or not the labels should be binarized, along with
		# the total number of classes
		self.batchSize = batchSize
		self.preprocessors = preprocessors
		self.aug = aug
		self.binarize = binarize
		self.classes = classes

		# open the HDF5 database for reading and determine the total
		# number of entries in the database
		self.db = h5py.File(dbPath)
		self.numImages = self.db["labels"].shape[0]

	def generator(self, passes=np.inf):
		# initialize the epoch count
		epochs = 0

		# keep looping infinitely -- the model will stop once we have
		# reach the desired number of epochs
		while epochs < passes:
			# loop over the HDF5 dataset
			for i in np.arange(0, self.numImages, self.batchSize):
				# extract the images and labels from the HDF dataset
				images = self.db["images"][i: i + self.batchSize]
				labels = self.db["labels"][i: i + self.batchSize]

				# check to see if the labels should be binarized
				if self.binarize:
					labels = np_utils.to_categorical(labels,
						self.classes)

				# check to see if our preprocessors are not None
				if self.preprocessors is not None:
					# initialize the list of processed images
					procImages = []

					# loop over the images
					for image in images:
						# loop over the preprocessors and apply each
						# to the image
						for p in self.preprocessors:
							image = p.preprocess(image)

						# update the list of processed images
						procImages.append(image)

					# update the images array to be the processed
					# images
					images = np.array(procImages)

				# if the data augmenator exists, apply it
				if self.aug is not None:
					(images, labels) = next(self.aug.flow(images,
						labels, batch_size=self.batchSize))

				# yield a tuple of images and labels
				yield (images, labels)

			# increment the total number of epochs
			epochs += 1

	def close(self):
		# close the database
		self.db.close()

AlexNet architecture. Why AlexNet is just because to use large size filters initially as we are upslacing most of the images to 256x256 in given dataset. In this case features size will be increased in the images and find those filters it would be easy if we use large size convolution filters.

In [45]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.advanced_activations import ELU
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.regularizers import l2
from keras import backend as K

class AlexNet:
	@staticmethod
	def build(width, height, depth, classes, reg=0.0002):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# Block #1: first CONV => RELU => POOL layer set
		model.add(Conv2D(96, (11, 11), strides=(4, 4),
			input_shape=inputShape, padding="same",
			kernel_regularizer=l2(reg)))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
		model.add(Dropout(0.25))

		# Block #2: second CONV => RELU => POOL layer set
		model.add(Conv2D(256, (5, 5), padding="same",
			kernel_regularizer=l2(reg)))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
		model.add(Dropout(0.25))

		# Block #3: CONV => RELU => CONV => RELU => CONV => RELU
		model.add(Conv2D(384, (3, 3), padding="same",
			kernel_regularizer=l2(reg)))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(384, (3, 3), padding="same",
			kernel_regularizer=l2(reg)))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(256, (3, 3), padding="same",
			kernel_regularizer=l2(reg)))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
		model.add(Dropout(0.25))

		# Block #4: first set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(4096, kernel_regularizer=l2(reg)))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))

		# Block #5: second set of FC => RELU layers
		model.add(Dense(4096, kernel_regularizer=l2(reg)))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes, kernel_regularizer=l2(reg)))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

Using class weights as data is having unbalced classification.
Images from train set 
MIDDLE - 10804
OLD - 2396
YOUNG - 6706

In [46]:
totals = [10804,  2396,  6706]
totals = np.array(totals)
classWeight = totals.max() / totals
print(classWeight)

[1.         4.50918197 1.61109454]


Interupting kernel or Ctrl + C ==> Stopping the training when found any over fitting to tweak the parameters to find optimal model.

In [47]:
checkpoints = "checkpoints" #path to output checkpoint directory
modelPath = "checkpoints/epoch_75.hdf5" #path to *specific* model checkpoint to load
start_epoch = 75 #epoch to restart training at

Training the model using fit_generator method

In [48]:
# import the necessary packages
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.models import load_model
from keras.optimizers import Adam
import json
import os

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=15, zoom_range=0.15,
	width_shift_range=0.15, height_shift_range=0.15, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")

# load the RGB means for the training set
means = json.loads(open(DATASET_MEAN).read())

# initialize the image preprocessors
sp = SimplePreprocessor(227, 227)
pp = PatchPreprocessor(227, 227)
mp = MeanPreprocessor(means["R"], means["G"], means["B"])
iap = ImageToArrayPreprocessor()

# initialize the training and validation dataset generators
trainGen = HDF5DatasetGenerator(TRAIN_HDF5, 128, aug=aug,
	preprocessors=[pp, mp, iap], classes=3)
valGen = HDF5DatasetGenerator(VAL_HDF5, 128,
	preprocessors=[sp, mp, iap], classes=3)

# initialize the optimizer
if modelPath is None:
	print("[INFO] compiling model...")
	opt = Adam(lr=1e-4)
	model = AlexNet.build(width=227, height=227, depth=3,
		classes=3, reg=0.00015)
	model.compile(loss="categorical_crossentropy", optimizer=opt,
		metrics=["accuracy"])
# otherwise, load the checkpoint from disk
else:
	print("[INFO] loading {}...".format(modelPath))
	model = load_model(modelPath)

	# update the learning rate
	print("[INFO] old learning rate: {}".format(
		K.get_value(model.optimizer.lr)))
	K.set_value(model.optimizer.lr, 1e-4)
	print("[INFO] new learning rate: {}".format(
		K.get_value(model.optimizer.lr)))   

# construct the set of callbacks
figPath = os.path.sep.join([OUTPUT_PATH,
	"alexnet_agepred.png"])
jsonPath = os.path.sep.join([OUTPUT_PATH,
	"alexnet_agepred.json"])
callbacks = [
	EpochCheckpoint(checkpoints, every=5,
		startAt=start_epoch),
	TrainingMonitor(figPath, jsonPath=jsonPath,
		startAt=start_epoch)]

# train the network
model.fit_generator(
	trainGen.generator(),
	steps_per_epoch=trainGen.numImages // 128,
	validation_data=valGen.generator(),
	validation_steps=valGen.numImages // 128,
	epochs=75,
	class_weight=classWeight,
	max_queue_size=10,
	callbacks=callbacks, verbose=1)

# save the model to file
print("[INFO] model is ready...")
#model.save(MODEL_PATH, overwrite=True)

# close the HDF5 datasets
trainGen.close()
valGen.close()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.


[INFO] loading checkpoints/epoch_75.hdf5...
[INFO] old learning rate: 9.999999747378752e-05
[INFO] new learning rate: 9.999999747378752e-05
Epoch 1/75
116/116 [==============================] - 196s 2s/step - loss: 0.9153 - accuracy: 0.8553 - val_loss: 0.8403 - val_accuracy: 0.8810
Epoch 2/75
116/116 [==============================] - 189s 2s/step - loss: 0.8855 - accuracy: 0.8654 - val_loss: 0.7601 - val_accuracy: 0.8971
Epoch 3/75
116/116 [==============================] - 188s 2s/step - loss: 0.8443 - accuracy: 0.8729 - val_loss: 0.9808 - val_accuracy: 0.8938
Epoch 4/75
116/116 [==============================] - 187s 2s/step - loss: 0.8274 - accuracy: 0.8801 - val_loss: 0.7393 - val_accuracy: 0.9000
Epoch 5/75
116/116 [==============================] - 186s 2s/step - loss: 0.8192 - accuracy: 0.8869 - val_loss: 0.9731 - val_accuracy: 0.8816
Epoch 6/75
116/116 [==============================] - 179s 2s/step - loss: 0.7984 - accuracy: 0.8857 - val_loss: 0.8625 - val_accuracy: 0.8878
Ep

KeyboardInterrupt: 

In [20]:
# save the model to file
print("[INFO] Serializing model...")
model.save(MODEL_PATH, overwrite=True)

[INFO] Serializing model...


Croping the image which is used for testing

In [49]:
# import the necessary packages
import numpy as np
import cv2

class CropPreprocessor:
	def __init__(self, width, height, horiz=True, inter=cv2.INTER_AREA):
		# store the target image width, height, whether or not
		# horizontal flips should be included, along with the
		# interpolation method used when resizing
		self.width = width
		self.height = height
		self.horiz = horiz
		self.inter = inter

	def preprocess(self, image):
		# initialize the list of crops
		crops = []

		# grab the width and height of the image then use these
		# dimensions to define the corners of the image based
		(h, w) = image.shape[:2]
		coords = [
			[0, 0, self.width, self.height],
			[w - self.width, 0, w, self.height],
			[w - self.width, h - self.height, w, h],
			[0, h - self.height, self.width, h]]

		# compute the center crop of the image as well
		dW = int(0.5 * (w - self.width))
		dH = int(0.5 * (h - self.height))
		coords.append([dW, dH, w - dW, h - dH])

		# loop over the coordinates, extract each of the crops,
		# and resize each of them to a fixed size
		for (startX, startY, endX, endY) in coords:
			crop = image[startY:endY, startX:endX]
			crop = cv2.resize(crop, (self.width, self.height),
				interpolation=self.inter)
			crops.append(crop)

		# check to see if the horizontal flips should be taken
		if self.horiz:
			# compute the horizontal mirror flips for each crop
			mirrors = [cv2.flip(c, 1) for c in crops]
			crops.extend(mirrors)

		# return the set of crops
		return np.array(crops)

Loading the best model from saved models and predict the test images with out using cropping

In [51]:
MODEL_TO_LOAD = "checkpoints/epoch_85.hdf5"
# import the necessary packages
from keras.models import load_model
import numpy as np
import progressbar
import json

# load the RGB means for the training set
means = json.loads(open(DATASET_MEAN).read())

# initialize the image preprocessors
sp = SimplePreprocessor(227, 227)
mp = MeanPreprocessor(means["R"], means["G"], means["B"])
cp = CropPreprocessor(227, 227)
iap = ImageToArrayPreprocessor()

# load the pretrained network
print("[INFO] loading model...")
model = load_model(MODEL_TO_LOAD)

# initialize the testing dataset generator, then make predictions on
# the testing data
print("[INFO] predicting on test data (no crops)...")
testGen = HDF5DatasetGenerator(TEST_HDF5, 128,
	preprocessors=[sp, mp, iap], classes=3)
predictions1 = model.predict_generator(testGen.generator(),
	steps=((testGen.numImages // 128)+1), max_queue_size=10)
testGen.close()

[INFO] loading model...
[INFO] predicting on test data (no crops)...


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.


In [52]:
len(list(predictions1))

6636

In [57]:
predictions1

array([[9.2111945e-01, 7.1310557e-02, 7.5700022e-03],
       [6.4193289e-04, 2.4859284e-04, 9.9910945e-01],
       [1.1953845e-01, 1.9540239e-02, 8.6092132e-01],
       ...,
       [9.6356964e-01, 3.6077190e-02, 3.5316986e-04],
       [7.0522183e-01, 2.9449651e-01, 2.8161146e-04],
       [7.3042780e-02, 8.3720600e-03, 9.1858518e-01]], dtype=float32)

Creating a list with predicted class label and corresponding test image ID, so that creating .csv file for submission will be easy.

In [53]:
from imutils import paths
testList = []
testList = [["Class","ID"]]
testPaths = [] #list(paths.list_images(TEST_IMAGES_PATH))
t = open(TEST_IMAGES_CSV)
t.__next__() # f.next() for Python 2.7
tlist = list(t) 
for i, row in enumerate(tlist):
	# extract the image and label from the row
	image = row.rstrip()
	imagepath = os.path.join(TEST_IMAGES_PATH, image)  
	testPaths.append(imagepath)
t.close()    

for testpath, pred in zip(testPaths, predictions1):
    testID = testpath.split(os.path.sep)[-1]
    pred = list(pred)
    res = pred.index(max(pred))
    if res == 0:
        res = "MIDDLE"
    elif res == 1:
        res = "OLD"
    elif res == 2:
        res = "YOUNG"
    
    testList.append([res,testID])

print(testList)
len(testList)

[['Class', 'ID'], ['MIDDLE', '25321.jpg'], ['YOUNG', '989.jpg'], ['YOUNG', '19277.jpg'], ['MIDDLE', '13093.jpg'], ['OLD', '5367.jpg'], ['MIDDLE', '19851.jpg'], ['YOUNG', '10384.jpg'], ['YOUNG', '24567.jpg'], ['MIDDLE', '2029.jpg'], ['MIDDLE', '4956.jpg'], ['MIDDLE', '26131.jpg'], ['MIDDLE', '17407.jpg'], ['MIDDLE', '11136.jpg'], ['MIDDLE', '23038.jpg'], ['YOUNG', '6333.jpg'], ['MIDDLE', '25076.jpg'], ['YOUNG', '22102.jpg'], ['OLD', '12396.jpg'], ['YOUNG', '24719.jpg'], ['MIDDLE', '21354.jpg'], ['YOUNG', '15362.jpg'], ['OLD', '10415.jpg'], ['YOUNG', '5295.jpg'], ['YOUNG', '5531.jpg'], ['MIDDLE', '16784.jpg'], ['MIDDLE', '22442.jpg'], ['MIDDLE', '16794.jpg'], ['MIDDLE', '11261.jpg'], ['MIDDLE', '11540.jpg'], ['MIDDLE', '24286.jpg'], ['OLD', '5997.jpg'], ['YOUNG', '4115.jpg'], ['YOUNG', '23605.jpg'], ['YOUNG', '5640.jpg'], ['MIDDLE', '22459.jpg'], ['MIDDLE', '24442.jpg'], ['YOUNG', '13300.jpg'], ['OLD', '23446.jpg'], ['YOUNG', '8907.jpg'], ['YOUNG', '9497.jpg'], ['OLD', '4530.jpg'], ['MID

6637

Creating results submission file for without crops

In [54]:
import csv
with open('submissionwocrop2.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(testList)

Loading best model to predict the data on cropped testset images as well. Usually accuracy will increase little bit.

In [55]:
MODEL_TO_LOAD = "checkpoints/epoch_85.hdf5"
# import the necessary packages
from keras.models import load_model
import numpy as np
import progressbar
import json

# load the RGB means for the training set
means = json.loads(open(DATASET_MEAN).read())

# initialize the image preprocessors
sp = SimplePreprocessor(227, 227)
mp = MeanPreprocessor(means["R"], means["G"], means["B"])
cp = CropPreprocessor(227, 227)
iap = ImageToArrayPreprocessor()

# load the pretrained network
print("[INFO] loading model...")
model = load_model(MODEL_TO_LOAD)
# re-initialize the testing set generator, this time excluding the
# `SimplePreprocessor`
testGen = HDF5DatasetGenerator(TEST_HDF5, 128,
	preprocessors=[], classes=3)
predictions = []

# initialize the progress bar
widgets = ["Evaluating: ", progressbar.Percentage(), " ",
	progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=testGen.numImages // 128,
	widgets=widgets).start()

# loop over a single pass of the test data
for (i, (images, labels)) in enumerate(testGen.generator(passes=1)):
	# loop over each of the individual images
	for image in images:
		# apply the crop preprocessor to the image to generate 10
		# separate crops, then convert them from images to arrays
		crops = cp.preprocess(image)
		crops = np.array([iap.preprocess(c) for c in crops],
			dtype="float32")

		# make predictions on the crops and then average them
		# together to obtain the final prediction
		pred = model.predict(crops)
		predictions.append(pred.mean(axis=0))

	# update the progress bar
	pbar.update(i)

# compute the rank-1 accuracy
pbar.finish()
testGen.close()

[INFO] loading model...


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
Evaluating: 100% |##############################################| Time: 0:02:17


In [56]:
len(predictions)

6636

In [58]:
predictions

[array([0.01541605, 0.3960703 , 0.5885136 ], dtype=float32),
 array([0.32383698, 0.19279069, 0.4833723 ], dtype=float32),
 array([0.40108055, 0.15339372, 0.44552574], dtype=float32),
 array([0.00521525, 0.00126205, 0.99352264], dtype=float32),
 array([0.26512548, 0.02977309, 0.7051014 ], dtype=float32),
 array([0.81072646, 0.01497848, 0.17429502], dtype=float32),
 array([0.20655885, 0.00361855, 0.7898226 ], dtype=float32),
 array([0.37714374, 0.00618002, 0.6166762 ], dtype=float32),
 array([0.00090736, 0.8678732 , 0.13121936], dtype=float32),
 array([0.9043134 , 0.00581478, 0.08987179], dtype=float32),
 array([9.9716836e-01, 3.7682959e-05, 2.7939845e-03], dtype=float32),
 array([0.6017748 , 0.24754545, 0.15067974], dtype=float32),
 array([0.92421836, 0.02646766, 0.04931402], dtype=float32),
 array([0.7857793 , 0.00447016, 0.20975056], dtype=float32),
 array([9.9780160e-01, 7.3641317e-04, 1.4620539e-03], dtype=float32),
 array([5.4362647e-02, 4.1345894e-04, 9.4522393e-01], dtype=float32

Creating a list with predicted class label and corresponding test image ID, so that creating .csv file for submission will be easy.

In [59]:
from imutils import paths
testList = [["Class","ID"]]

testPaths = [] #list(paths.list_images(TEST_IMAGES_PATH))
t = open(TEST_IMAGES_CSV)
t.__next__() # f.next() for Python 2.7
tlist = list(t) 
for i, row in enumerate(tlist):
	# extract the image and label from the row
	image = row.rstrip()
	imagepath = os.path.join(TEST_IMAGES_PATH, image)  
	testPaths.append(imagepath)
t.close()    

for testpath, pred in zip(testPaths, predictions):
    testID = testpath.split(os.path.sep)[-1]
    pred = list(pred)
    res = pred.index(max(pred))
    if res == 0:
        res = "MIDDLE"
    elif res == 1:
        res = "OLD"
    elif res == 2:
        res = "YOUNG"
    
    testList.append([res,testID])

print(testList)
len(testList)

[['Class', 'ID'], ['YOUNG', '25321.jpg'], ['YOUNG', '989.jpg'], ['YOUNG', '19277.jpg'], ['YOUNG', '13093.jpg'], ['YOUNG', '5367.jpg'], ['MIDDLE', '19851.jpg'], ['YOUNG', '10384.jpg'], ['YOUNG', '24567.jpg'], ['OLD', '2029.jpg'], ['MIDDLE', '4956.jpg'], ['MIDDLE', '26131.jpg'], ['MIDDLE', '17407.jpg'], ['MIDDLE', '11136.jpg'], ['MIDDLE', '23038.jpg'], ['MIDDLE', '6333.jpg'], ['YOUNG', '25076.jpg'], ['YOUNG', '22102.jpg'], ['MIDDLE', '12396.jpg'], ['MIDDLE', '24719.jpg'], ['YOUNG', '21354.jpg'], ['YOUNG', '15362.jpg'], ['YOUNG', '10415.jpg'], ['YOUNG', '5295.jpg'], ['MIDDLE', '5531.jpg'], ['YOUNG', '16784.jpg'], ['YOUNG', '22442.jpg'], ['YOUNG', '16794.jpg'], ['OLD', '11261.jpg'], ['MIDDLE', '11540.jpg'], ['YOUNG', '24286.jpg'], ['YOUNG', '5997.jpg'], ['YOUNG', '4115.jpg'], ['YOUNG', '23605.jpg'], ['MIDDLE', '5640.jpg'], ['YOUNG', '22459.jpg'], ['MIDDLE', '24442.jpg'], ['YOUNG', '13300.jpg'], ['YOUNG', '23446.jpg'], ['MIDDLE', '8907.jpg'], ['MIDDLE', '9497.jpg'], ['YOUNG', '4530.jpg'], [

6637

Creating test results in .csv submission file with crops. Usaully accruacy will increase slight than without crops prediction.

In [60]:
import csv
with open('submissionwithcrop2.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(testList)